# VisualOdometrySLAM

In [47]:
import random

import cv2
import numpy as np
import pandas as pd

def seed_all(seed: int) -> None:
    np.random.seed(seed)
    random.seed(seed)
    cv2.setRNGSeed(seed)

In [48]:
SEED = 42
seed_all(SEED)

In [49]:
from pathlib import Path

DATA_PATH = Path('../data/')
DATA_PATH.mkdir(parents=True, exist_ok=True)

DATA_PATH_VIDEO = DATA_PATH /Path('visual_odometry/')
DATA_PATH_VIDEO.mkdir(parents=True, exist_ok=True)

DATA_PATH_OUTPUT = DATA_PATH / Path('output_data/')
DATA_PATH_OUTPUT.mkdir(parents=True, exist_ok=True)

DATA_PATH_SAVE_MODELS = DATA_PATH / Path('models/')
DATA_PATH_SAVE_MODELS.mkdir(parents=True, exist_ok=True)

DATA_IMGS = Path('../imgs')
DATA_IMGS.mkdir(parents=True, exist_ok=True)

In [50]:
import sys
import os

project_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_path)

## Общие методы

| Параметр                  | Где в YAML                     | Что означает                                  | Как определить / откуда берётся                 |
| ------------------------- | ------------------------------ | --------------------------------------------- | ----------------------------------------------- |
| `fx`, `fy`                | `camera_matrix.data[0]`, `[4]` | Фокусное расстояние по осям X и Y             | Из калибровки (например, `cv2.calibrateCamera`) |
| `cx`, `cy`                | `camera_matrix.data[2]`, `[5]` | Координаты главной точки (оптический центр)   | Центр кадра или из калибровки                   |
| `camera_matrix`           | `camera_matrix.data`           | Внутренняя матрица 3×3                        | Результат калибровки                            |
| `distortion_model`        | `distortion_model`             | Модель дисторсии (`plumb_bob`, `equidistant`) | Выбирается при калибровке, по типу объектива    |
| `distortion_coefficients` | `distortion_coefficients.data` | Коэффициенты искажений (k1, k2, p1, p2, k3)   | Выход `cv2.calibrateCamera`, `Kalibr`, ROS      |
| `image_width` / `height`  | `image_width`, `image_height`  | Размер изображения (в пикселях)               | Известно по сенсору или из примера калибровки   |
| `camera_name`             | `camera_name`                  | Название камеры                               | Задаётся вручную (опционально)                  |


In [51]:
import cv2
import numpy as np
from pathlib import Path
from typing import Union, List


def load_video_frames(video_path: Union[str, Path], target_fps: float) -> List[np.ndarray]:
    """
    Извлекает кадры из видео с заданной частотой и переводит их в оттенки серого.

    :param video_path: путь к видеофайлу
    :param target_fps: целевая частота обработки кадров
    :return: список кадров (в оттенках серого)
    """
    video_path = Path(video_path)
    if not video_path.exists():
        raise FileNotFoundError(f'Видео не найдено по пути: {video_path.resolve()}')

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise IOError(f'Не удалось открыть видеофайл: {video_path}')

    video_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(video_fps / target_fps)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration_sec = total_frames / video_fps
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    print(f"Информация о видео:")
    print(f"- Имя файла: {video_path.name}")
    print(f"- Размер кадра: {frame_width} x {frame_height}")
    print(f"- Оригинальный FPS: {video_fps:.2f}")
    print(f"- Общее число кадров: {total_frames}")
    print(f"- Длительность видео: {duration_sec:.2f} секунд")
    print(f"- Целевая частота обработки: {target_fps} FPS")
    print(f"- Будет обрабатываться каждый {frame_interval}-й кадр")

    processed_frames = []
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % frame_interval == 0:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            processed_frames.append(gray)

        frame_idx += 1

    cap.release()
    print(f"Обработано кадров: {len(processed_frames)} при целевом FPS: {target_fps}")
    return processed_frames


def load_kitti_sequence(path: Union[str, Path], grayscale: bool = True) -> List[np.ndarray]:
    image_paths = sorted(Path(path).glob("*.png"))
    frames = []
    for p in image_paths:
        img = cv2.imread(str(p), cv2.IMREAD_GRAYSCALE if grayscale else cv2.IMREAD_COLOR)
        if img is not None:
            frames.append(img)
    return frames


## ORB params

| Параметр        | Тип     | Значение по умолчанию  | Описание и рекомендации                                                                                                         |
| --------------- | ------- | ---------------------- | ------------------------------------------------------------------------------------------------------------------------------- |
| `nfeatures`     | `int`   | 500                    | Максимальное количество фич, которые ORB попытается найти. <br>🔧 Увеличь до 3000–5000 для богатой сцены или сеточной детекции. |
| `scaleFactor`   | `float` | 1.2                    | Масштаб между уровнями пирамиды. <br>Меньше — больше перекрытий, лучше для масштабных сцен, но медленнее.                       |
| `nlevels`       | `int`   | 8                      | Кол-во уровней в пирамиде изображений. <br>🔧 Увеличь до 12–16, если ожидается масштабный диапазон объектов.                    |
| `edgeThreshold` | `int`   | 31                     | Отступ от границ изображения. <br>🔧 Уменьши до 10–15, если хочешь находить фичи у краёв.                                       |
| `firstLevel`    | `int`   | 0                      | Начальный уровень пирамиды (обычно 0). Почти никогда не меняется.                                                               |
| `WTA_K`         | `int`   | 2                      | Метод генерации BRIEF-дескриптора: 2 = быстрее, 3 или 4 = точнее. <br>Используй 2, если нет проблем с точностью.                |
| `scoreType`     | `int`   | `cv2.ORB_HARRIS_SCORE` | Выбор способа ранжирования углов: `cv2.ORB_HARRIS_SCORE` или `cv2.ORB_FAST_SCORE`. <br>`HARRIS` точнее, `FAST` — быстрее.       |
| `patchSize`     | `int`   | 31                     | Размер области вокруг фичи для дескриптора. <br>🔧 31 — стандарт, можно увеличить до 49, если объект крупный.                   |
| `fastThreshold` | `int`   | 20                     | Порог FAST-угла: чем меньше — тем больше фич. <br>🔧 Снизь до 5–10, если ORB пропускает фичи.                                   |


In [52]:
import yaml
from pathlib import Path
from collections import Counter
from typing import Union, List, Optional

import cv2
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt



In [53]:
class MapPoint:
    def __init__(self, coord_3d: np.ndarray, descriptor: np.ndarray):
        self.coord = coord_3d  # (X, Y, Z)
        self.descriptor = descriptor
        self.observations = []  # список (frame_idx, keypoint_idx, (x, y))


class KeyFrame:
    def __init__(self, frame_id: int, image: np.ndarray, keypoints, descriptors, pose: np.ndarray):
        self.id = frame_id
        self.image = image
        self.keypoints = keypoints
        self.descriptors = descriptors
        self.pose = pose        

In [54]:
from typing import List, Tuple
import numpy as np
import cv2


class MatcherFilterMixin:
    @staticmethod
    def filter_by_ratio_test(knn_matches, kp1, kp2, ratio_thresh):
        good_matches = []
        pts1, pts2 = [], []

        for pair in knn_matches:
            if len(pair) < 2:
                continue
            m, n = pair
            if m.distance < ratio_thresh * n.distance:
                good_matches.append(m)
                pts1.append(kp1[m.queryIdx].pt)
                pts2.append(kp2[m.trainIdx].pt)

        return good_matches, np.float32(pts1), np.float32(pts2)

    @staticmethod
    def filter_by_max_distance(matches, pts1, pts2, max_dist):
        filtered_matches = []
        filtered_pts1, filtered_pts2 = [], []

        for i, m in enumerate(matches):
            if m.distance < max_dist:
                filtered_matches.append(m)
                filtered_pts1.append(pts1[i])
                filtered_pts2.append(pts2[i])

        return filtered_matches, np.float32(filtered_pts1), np.float32(filtered_pts2)

    @staticmethod
    def filter_unique_matches(matches, pts1, pts2):
        seen_trainIdx = set()
        unique_matches = []
        unique_pts1, unique_pts2 = [], []

        for i, m in enumerate(matches):
            if m.trainIdx not in seen_trainIdx:
                seen_trainIdx.add(m.trainIdx)
                unique_matches.append(m)
                unique_pts1.append(pts1[i])
                unique_pts2.append(pts2[i])

        return unique_matches, np.float32(unique_pts1), np.float32(unique_pts2)

    @staticmethod
    def filter_by_ransac_fundamental(matches, pts1, pts2):
        if len(matches) < 8:
            return [], np.array([]), np.array([])

        F, mask = cv2.findFundamentalMat(pts1, pts2, method=cv2.FM_RANSAC, ransacReprojThreshold=1.0)

        inlier_matches = []
        inlier_pts1, inlier_pts2 = [], []

        if mask is not None:
            for i, m in enumerate(matches):
                if mask[i]:
                    inlier_matches.append(m)
                    inlier_pts1.append(pts1[i])
                    inlier_pts2.append(pts2[i])

        return inlier_matches, np.float32(inlier_pts1), np.float32(inlier_pts2)

    @staticmethod
    def filter_by_exclusion_mask(matches, pts1, pts2, mask_exclude_regions: List[Tuple[int, int, int, int]]):
        if not mask_exclude_regions:
            return matches, pts1, pts2

        filtered_matches = []
        filtered_pts1 = []
        filtered_pts2 = []

        for i, m in enumerate(matches):
            x1, y1 = pts1[i]
            x2, y2 = pts2[i]

            excluded = any(
                (xmin <= x1 <= xmax and ymin <= y1 <= ymax) or
                (xmin <= x2 <= xmax and ymin <= y2 <= ymax)
                for (xmin, ymin, xmax, ymax) in mask_exclude_regions
            )

            if not excluded:
                filtered_matches.append(m)
                filtered_pts1.append((x1, y1))
                filtered_pts2.append((x2, y2))

        return filtered_matches, np.float32(filtered_pts1), np.float32(filtered_pts2)

    @staticmethod
    def filter_mutual_matches(
        matcher: cv2.DescriptorMatcher,
        des1: np.ndarray,
        des2: np.ndarray,
        kp1: List[cv2.KeyPoint],
        kp2: List[cv2.KeyPoint],
        matches: List[cv2.DMatch],
        pts1: np.ndarray,
        pts2: np.ndarray
    ) -> Tuple[List[cv2.DMatch], np.ndarray, np.ndarray]:
        """
        Оставляет только взаимные совпадения (A→B и B→A).
        
        :param matcher: объект cv2.DescriptorMatcher (например, BFMatcher или FlannBasedMatcher)
        :param des1: дескрипторы первого изображения
        :param des2: дескрипторы второго изображения
        :param kp1: ключевые точки первого изображения
        :param kp2: ключевые точки второго изображения
        :param matches: прямые совпадения A→B
        :param pts1: координаты точек первого изображения
        :param pts2: координаты точек второго изображения
        :return: (взаимные совпадения, pts1, pts2)
        """
        reverse_matches = matcher.match(des2, des1)
        reverse_set = {(m.trainIdx, m.queryIdx) for m in reverse_matches}

        mutual_matches = []
        mutual_pts1, mutual_pts2 = [], []

        for i, m in enumerate(matches):
            if (m.queryIdx, m.trainIdx) in reverse_set:
                mutual_matches.append(m)
                mutual_pts1.append(pts1[i])
                mutual_pts2.append(pts2[i])

        return mutual_matches, np.float32(mutual_pts1), np.float32(mutual_pts2)

In [ ]:
from regex import F


class VisualOdometry(MatcherFilterMixin):
    def __init__(self, 
                 video_path: Union[str, Path],
                 calibration_file: Union[str, Path],                   
                 **params) -> None:
        """
        Инициализация пайплайна визуальной одометрии.

        Загружает калибровку камеры и видео, а также позволяет задать параметры ORB-детектора через `params`.

        :param video_path: путь к видеофайлу (например, .mp4)
        :param calibration_file: путь к YAML-файлу с калибровкой (формат OpenCV / ROS)
        :param target_fps: частота обработки кадров
        :param params: дополнительные параметры, передаваемые в ORB.create(), например:
                       nfeatures=3000, edgeThreshold=10, fastThreshold=5 и т.п.
        """
        self.params: dict = params
        self.video_path = Path(video_path)
        self.calibration_file = Path(calibration_file)
        self.target_fps = self.params.get("target_fps", 2)
        
        self.K: np.ndarray 
        self.distCoeffs: Optional[np.ndarray]
        self.distortion_model: Optional[str]

        self.K, self.distCoeffs, self.distortion_model = self.load_calibration(self.calibration_file)
        self.frames: List[np.ndarray] = self.load_frames(self.video_path)
        
        self.keypoints: List = []  
        self.matches: List = []   
        self.keyframes: List[KeyFrame] = []
        self.mappoints: List[MapPoint] = []
        self.triangulated_points: List[np.ndarray] = []  
        self.poses: List[np.ndarray] = []             

        self.orb = self.create_orb()
        self.matcher = self.create_matcher()
        
    def load_calibration(self, path: Union[str, Path]) -> Tuple[np.ndarray, Optional[np.ndarray], Optional[str]]:
        """
        Универсальная загрузка калибровки:
        - Если .txt → KITTI формат (с префиксом P0: или без)
        - Если .yaml → OpenCV / ROS
        """
        path = Path(path)
        if not path.exists():
            raise FileNotFoundError(f"Файл калибровки не найден: {path.resolve()}")

        if path.suffix == ".txt":
            with path.open("r") as f:
                lines = [line.strip() for line in f if line.strip()]

            # Попробуем сначала с P0:/P1:
            for line in lines:
                if line.startswith("P0:") or line.startswith("P1:"):
                    values = list(map(float, line.split()[1:]))
                    if len(values) == 12:
                        P = np.array(values).reshape(3, 4)
                        return P[:, :3], None, "none"

            # Попробуем первую строку как просто 12 чисел
            for line in lines:
                values = list(map(float, line.split()))
                if len(values) == 12:
                    P = np.array(values).reshape(3, 4)
                    return P[:, :3], None, "none"

            raise ValueError("Неверный формат KITTI-файла. Ожидалась строка из 12 чисел.")

        elif path.suffix in (".yaml", ".yml"):
            import yaml
            with path.open('r') as f:
                data = yaml.safe_load(f)

            K_data = data["camera_matrix"]["data"]
            K = np.array(K_data, dtype=np.float64).reshape(3, 3)

            dist_data = data.get("distortion_coefficients", {}).get("data", [])
            distCoeffs = np.array(dist_data, dtype=np.float64) if dist_data else None

            model = data.get("distortion_model", None)
            return K, distCoeffs, model

        else:
            raise ValueError(f"Неизвестный формат файла калибровки: {path}") 

    def load_frames(self, path: Union[str, Path]) -> List[np.ndarray]:
        """
        Загружает последовательность кадров:
        - если путь — директория, то PNG (KITTI)
        - если файл — то видео с заданной частотой
        Все кадры приводятся к оттенкам серого.
        """
        path = Path(path)
        frames = []

        if path.is_dir():
            # KITTI-подобная последовательность
            image_paths = sorted(path.glob("*.png"))
            if not image_paths:
                raise FileNotFoundError(f"Не найдено изображений в директории: {path.resolve()}")

            for p in image_paths:
                img = cv2.imread(str(p), cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    frames.append(img)

            print(f"[KITTI] Загружено {len(frames)} кадров из {path.name}")

        else:
            # Видеофайл
            if not path.exists():
                raise FileNotFoundError(f"Видео не найдено: {path.resolve()}")

            cap = cv2.VideoCapture(str(path))
            if not cap.isOpened():
                raise IOError(f"Не удалось открыть видео: {path}")

            video_fps = cap.get(cv2.CAP_PROP_FPS)
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            frame_interval = max(1, int(video_fps / self.target_fps)) if video_fps else 1

            print(f"[Видео] Имя: {path.name}")
            print(f"- FPS: {video_fps:.2f}, кадры: {total_frames}, интервал: {frame_interval}")

            frame_idx = 0
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                if frame_idx % frame_interval == 0:
                    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    frames.append(gray)
                frame_idx += 1

            cap.release()
            print(f"[Видео] Загружено {len(frames)} кадров с шагом {frame_interval}")

        if not frames:
            raise RuntimeError("Не удалось загрузить ни одного кадра.")
        return frames

    def clear_data(self):
        """
        Очищает все внутренние списки и данные.
        Используется для перезапуска пайплайна.
        """
        self.keypoints.clear()
        self.matches.clear()
        self.keyframes.clear()
        self.mappoints.clear()
        self.poses.clear()
        self.triangulated_points.clear()

    # def create_matcher(self) -> cv2.DescriptorMatcher:
    #     """
    #     Создаёт и возвращает подходящий матчер дескрипторов для ORB.
    #     :return: объект cv2.DescriptorMatcher
    #     """
    #     FLANN_INDEX_LSH = 6
    #     index_params = dict(
    #         algorithm=FLANN_INDEX_LSH,
    #         table_number=self.params.get("flann_table_number", 6),
    #         key_size=self.params.get("flann_key_size", 12), 
    #         multi_probe_level=self.params.get("flann_multi_probe_level", 1)
    #     )
    #     search_params = dict(
    #         checks=self.params.get("flann_checks", 50)
    #     )
    #     return cv2.FlannBasedMatcher(index_params, search_params)
    
    def create_matcher(self):
        return cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    
    def create_orb(self) -> cv2.ORB:
        """
        Создаёт ORB-детектор с параметрами из конструктора.
        :return: объект cv2.ORB
        """
        orb_params = {
            "nfeatures": self.params.get("nfeatures", 1000),
            "scaleFactor": self.params.get("scaleFactor", 1.2),
            "nlevels": self.params.get("nlevels", 8),
            "edgeThreshold": self.params.get("edgeThreshold", 31),
            "fastThreshold": self.params.get("fastThreshold", 20)
        }
        return cv2.ORB.create(**orb_params)
    
    def detect_and_compute(self, image: np.ndarray) -> Tuple[List[cv2.KeyPoint], np.ndarray]:
        use_grid = self.params.get("use_grid", False)

        if not use_grid:
            return self.orb.detectAndCompute(image, None)

        num_rows = self.params.get("grid_rows", 8)
        num_cols = self.params.get("grid_cols", 8)

        h, w = image.shape
        kp_all, des_all = [], []

        for i in range(num_rows):
            for j in range(num_cols):
                x0 = int(j * w / num_cols)
                x1 = int((j + 1) * w / num_cols)
                y0 = int(i * h / num_rows)
                y1 = int((i + 1) * h / num_rows)

                roi = image[y0:y1, x0:x1]
                kp, des = self.orb.detectAndCompute(roi, None)

                for k in kp:
                    k.pt = (k.pt[0] + x0, k.pt[1] + y0)

                if kp:
                    kp_all.extend(kp)
                    if des is not None:
                        des_all.append(des)

        if des_all:
            des_all = np.vstack(des_all)
        else:
            des_all = None
        print(f"[Debug] Найдено {len(kp_all)} ключевых точек")
        return kp_all, des_all
    
    def add_keyframe(self, frame_id: int, image: np.ndarray, keypoints, descriptors, pose: np.ndarray):
        kf = KeyFrame(frame_id, image, keypoints, descriptors, pose)
        self.keyframes.append(kf)

    def add_mappoint(self, coord_3d: np.ndarray, frame_id: int, kp_idx: int, pt2d: Tuple[float, float], descriptor: np.ndarray):
        mp = MapPoint(coord_3d, descriptor)
        mp.observations.append((frame_id, kp_idx, pt2d))
        self.mappoints.append(mp)
           
    def get_matches(
        self,
        kp1: List[cv2.KeyPoint],
        des1: np.ndarray,
        kp2: List[cv2.KeyPoint],
        des2: np.ndarray
    ) -> Tuple[List[cv2.KeyPoint], List[cv2.KeyPoint], List[cv2.DMatch], np.ndarray, np.ndarray]:

        if des1 is None or des2 is None or len(kp1) == 0 or len(kp2) == 0:
            return [], [], [], np.array([]), np.array([])

        # 1. Первичное сопоставление дескрипторов
        knn_matches = self.matcher.knnMatch(des1, des2, k=2)

        # 2. Lowe's ratio test
        ratio_thresh = self.params.get("ratio_thresh", 0.6)
        good_matches, pts1, pts2 = self.filter_by_ratio_test(knn_matches, kp1, kp2, ratio_thresh)

        # 3. Ограничение по дистанции
        max_dist = self.params.get("max_match_distance", 60)
        good_matches, pts1, pts2 = self.filter_by_max_distance(good_matches, pts1, pts2, max_dist)

        # 4. Удаление повторяющихся trainIdx
        good_matches, pts1, pts2 = self.filter_unique_matches(good_matches, pts1, pts2)

        # 5. Геометрическая фильтрация через фундаментальную матрицу
        good_matches, pts1, pts2 = self.filter_by_ransac_fundamental(good_matches, pts1, pts2)

        # # 6. Исключение по зонам маски интереса (если указаны)
        # mask_exclude_regions = self.params.get("mask_exclude_regions", [])
        # good_matches, pts1, pts2 = self.filter_by_exclusion_mask(good_matches, pts1, pts2, mask_exclude_regions)

        # # 7. Взаимные совпадения A→B и B→A
        # if self.params.get("use_mutual_check", False):
        #     good_matches, pts1, pts2 = self.filter_mutual_matches(
        #         des1, des2, kp1, kp2, good_matches, pts1, pts2
        #     )
        
        self.keypoints.append((kp1, kp2))
        self.matches.append(good_matches)

        return kp1, kp2, good_matches, pts1, pts2
    
    def estimate_pose_opencv(self, pts1: np.ndarray, pts2: np.ndarray) -> Optional[Tuple[np.ndarray, np.ndarray]]:
        """
        Упрощённая оценка движения камеры на основе встроенной функции OpenCV.

        :param pts1: координаты точек в первом изображении (Nx2)
        :param pts2: координаты точек во втором изображении (Nx2)
        :return: кортеж (R, t) — матрица поворота и вектор смещения; либо None при ошибке
        """
        if pts1.shape[0] < 8 or pts2.shape[0] < 8:
            print("Недостаточно точек для оценки движения.")
            return None
        E, mask = cv2.findEssentialMat(
            pts1, pts2, self.K, 
            method=cv2.RANSAC, 
            prob=0.999, 
            threshold=1.0
        )
        if E is None or mask is None or mask.sum() < 8:
            print("Не удалось вычислить матрицу Essential.")
            return None
        _, R, t, pose_mask = cv2.recoverPose(E, pts1, pts2, self.K, mask=mask)
        
        t = t / np.linalg.norm(t)
        
        T = np.eye(4)
        T[:3, :3] = R
        T[:3, 3] = t.ravel()
        return T
    
    @staticmethod
    def _form_transformation_matrix(R: np.ndarray, t: np.ndarray) -> np.ndarray:
        """
        Формирует 4×4 однородную матрицу трансформации из R и t.

        :param R: матрица поворота 3×3
        :param t: вектор смещения (3,)
        :return: 4×4 матрица T
        """
        T = np.eye(4, dtype=np.float64)
        T[:3, :3] = R
        T[:3, 3] = t
        return T

    def decompose_essential_custom(self, E: np.ndarray, q1: np.ndarray, q2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """
        Декомпозирует Essential-матрицу, выбирает правильную пару (R, t),
        проверяя, сколько точек оказываются перед камерами (cheirality check).
        Также вычисляет относительный масштаб.

        :param E: матрица Essential (3×3)
        :param q1: сопоставленные точки в кадре i-1
        :param q2: сопоставленные точки в кадре i
        :return: правильные (R, t)
        """

        def triangulate_and_score(R, t):
            # Формируем 4×4 трансформацию
            T = self._form_transformation_matrix(R, t)

            # Камера 1: [K | 0], Камера 2: K * [R | t]
            P0 = self.K @ np.hstack((np.eye(3), np.zeros((3, 1))))
            P1 = self.K @ T[:3, :]

            # Триангуляция
            points_4d_hom = cv2.triangulatePoints(P0, P1, q1.T, q2.T)  # 4×N
            points_3d_1 = points_4d_hom[:3, :] / points_4d_hom[3, :]

            # Преобразуем точки в систему второй камеры
            points_4d_cam2 = T @ points_4d_hom
            points_3d_2 = points_4d_cam2[:3, :] / points_4d_cam2[3, :]

            # cheirality: сколько точек с положительным z в обеих камерах
            z1 = points_3d_1[2, :] > 0
            z2 = points_3d_2[2, :] > 0
            valid = z1 & z2
            num_valid = np.sum(valid)

            # относительный масштаб (по длине векторов между соседними точками)
            if np.sum(valid) >= 2:
                dist1 = np.linalg.norm(np.diff(points_3d_1.T[valid], axis=0), axis=1)
                dist2 = np.linalg.norm(np.diff(points_3d_2.T[valid], axis=0), axis=1)
                scale = np.mean(dist1 / dist2)
            else:
                scale = 1.0  # fallback

            return num_valid, scale

        # Получаем 4 возможные пары R, t
        R1, R2, t = cv2.decomposeEssentialMat(E)
        t = np.squeeze(t)
        candidates = [(R1,  t), (R1, -t), (R2,  t), (R2, -t)]

        # Проверяем все и выбираем ту, где больше точек спроецированы вперед
        scores = []
        scales = []
        for R, t_candidate in candidates:
            score, scale = triangulate_and_score(R, t_candidate)
            scores.append(score)
            scales.append(scale)

        best_idx = np.argmax(scores)
        best_R, best_t = candidates[best_idx]
        best_scale = scales[best_idx]

        # Применим относительный масштаб
        t = best_t * best_scale

        # Нормализация и установка фиксированного масштаба
        desired_scale = 1
        t = t / np.linalg.norm(t) * desired_scale
        return best_R, t
    
    def estimate_pose_custom(self, pts1: np.ndarray, pts2: np.ndarray) -> Optional[np.ndarray]:
        """
        Оценивает трансформацию (R, t) между двумя изображениями вручную:
        - извлекает матрицу Essential;
        - выполняет собственную декомпозицию;
        - строит 4×4 матрицу преобразования.

        :param pts1: координаты сопоставленных точек на кадре i-1 (Nx2)
        :param pts2: координаты сопоставленных точек на кадре i   (Nx2)
        :return: 4×4 матрица трансформации (или None при неудаче)
        """
        if pts1.shape[0] < 8 or pts2.shape[0] < 8:
            print("Недостаточно точек для вычисления матрицы Essential.")
            return None

        E, mask = cv2.findEssentialMat(pts1, pts2, self.K, method=cv2.RANSAC, threshold=1.0, prob=0.999)
        if E is None:
            print("Не удалось найти матрицу Essential.")
            return None

        R, t = self.decompose_essential_custom(E, pts1, pts2)
        T = self._form_transformation_matrix(R, np.squeeze(t))
        return T

    @staticmethod
    def filter_pointcloud(points: np.ndarray,
                        z_min: float = 0.1,
                        z_max: float = 10.0,
                        x_range: Optional[Tuple[float, float]] = None,
                        y_range: Optional[Tuple[float, float]] = None,
                        voxel_size: Optional[float] = 0.05,
                        apply_statistical: bool = True,
                        nb_neighbors: int = 20,
                        std_ratio: float = 2.0) -> np.ndarray:
        """
        Фильтрует облако точек на основе геометрических и статистических критериев.

        :param points: Массив 3D-точек формы (N, 3)
        
        :param z_min: Минимально допустимое значение глубины Z. Удаляются точки ближе, чем z_min.
        :param z_max: Максимально допустимая глубина Z. Удаляются слишком дальние точки.
        
        :param x_range: Кортеж (min_x, max_x), ограничивающий координаты X. Если None — не фильтруется.
        :param y_range: Кортеж (min_y, max_y), ограничивающий координаты Y. Если None — не фильтруется.
        
        :param voxel_size: Размер ячейки в voxel grid фильтрации. Если None — voxel-фильтрация не применяется.
                        Используется для упрощения и разреживания точек (например, 0.1 = 5 см).
        
        :param nb_neighbors: Число соседей для оценки плотности (используется при статистической фильтрации).
        :param std_ratio: Порог удаления выбросов — точки, чья средняя дистанция отклоняется на std_ratio,
                        будут удалены.
        
        :return: Отфильтрованное облако точек (M, 3)
        """
        # Фильтрация по глубине и NaN/Inf
        mask = (points[:, 2] > z_min) & (points[:, 2] < z_max)
        mask &= np.isfinite(points).all(axis=1)

        # Фильтрация по X, Y (если заданы)
        if x_range:
            mask &= (points[:, 0] > x_range[0]) & (points[:, 0] < x_range[1])
        if y_range:
            mask &= (points[:, 1] > y_range[0]) & (points[:, 1] < y_range[1])

        # Применение маски
        points = points[mask]
        if len(points) == 0:
            return points

        # Конвертация в формат Open3D
        import open3d as o3d
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)

        # Voxel Grid — упрощение
        if voxel_size:
            pcd = pcd.voxel_down_sample(voxel_size=voxel_size)

        # Статистическая фильтрация выбросов
        if apply_statistical:
            pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)

        return np.asarray(pcd.points)

    def triangulate_points(self,
                        pts1: np.ndarray,
                        pts2: np.ndarray,
                        T1: np.ndarray,
                        T2: np.ndarray) -> Optional[np.ndarray]:
        """
        Выполняет триангуляцию 3D-точек из пары кадров, возвращает точки в мировой системе координат.

        :param pts1: точки в первом изображении (Nx2)
        :param pts2: точки во втором изображении (Nx2)
        :param T1: глобальная поза первой камеры (4×4)
        :param T2: глобальная поза второй камеры (4×4)
        :return: массив 3D-точек в мировой системе (M×3), или None
        """
        if pts1.shape[0] < 8 or pts2.shape[0] < 8:
            return None

        P1 = self.K @ T1[:3, :]
        P2 = self.K @ T2[:3, :]


        points_4d = cv2.triangulatePoints(P1, P2, pts1.T, pts2.T)  # 4×N
        points_3d = points_4d[:3, :] / points_4d[3, :]             # 3×N → декартовы координаты
        points_3d = points_3d.T                                     # N×3

        filtered_points = self.filter_pointcloud(
            points_3d,
            z_min=0.01,
            z_max=200.0,
            voxel_size=None,
            apply_statistical=True
        )

        return filtered_points if len(filtered_points) > 0 else None
    
    def visualize_matches(self, delay_ms: int = 1000):
        num_pairs = len(self.keypoints)
        if num_pairs == 0:
            print("Нет сопоставлений. Сначала вызови run().")
            return

        print("Управление: [A] назад, [D] вперёд, [Q] выход")

        i = 0
        while True:
            kp1, kp2 = self.keypoints[i]
            matches = self.matches[i]
            img1 = self.frames[i]
            img2 = self.frames[i + 1]

            img1_color = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
            img2_color = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

            vis = cv2.drawMatches(
                img1_color, kp1,
                img2_color, kp2,
                matches, None,
                matchColor=(0, 255, 0),
                singlePointColor=(255, 0, 0),
                flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
            )

            # Отрисовать текущий индекс
            text = f"Frame pair: {i} / {num_pairs - 1}"
            cv2.putText(vis, text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)

            cv2.imshow("Feature Matches (A/D/Q)", vis)
            key = cv2.waitKey(delay_ms) & 0xFF

            if key == ord('q'):
                print("Выход из визуализации.")
                break
            elif key == ord('d'):
                i = min(i + 1, num_pairs - 1)
            elif key == ord('a'):
                i = max(i - 1, 0)

        cv2.destroyAllWindows()

    def visualize_keypoints_pairwise(self, delay_ms: int = 0):
        """
        Визуализирует пары кадров с нанесёнными ORB-фичами без соединения линиями.

        Управление:
            [A] — назад
            [D] — вперёд
            [Q] — выход
        """
        num_pairs = len(self.keypoints)
        if num_pairs == 0:
            print("Нет сохранённых ключевых точек. Сначала вызови run().")
            return

        print("Управление: [A] назад, [D] вперёд, [Q] выход")

        i = 0
        while True:
            kp1, kp2 = self.keypoints[i]
            img1 = self.frames[i]
            img2 = self.frames[i + 1]

            img1_color = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
            img2_color = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

            vis1 = cv2.drawKeypoints(img1_color, kp1, None, color=(0, 255, 0), flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)
            vis2 = cv2.drawKeypoints(img2_color, kp2, None, color=(255, 0, 0), flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)

            # Объединение изображений
            vis = np.hstack((vis1, vis2))

            # Подпись
            text = f"Frame pair: {i} / {num_pairs - 1}   Points: {len(kp1)} / {len(kp2)}"
            cv2.putText(vis, text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)

            cv2.imshow("Keypoints Only (A/D/Q)", vis)
            key = cv2.waitKey(delay_ms) & 0xFF

            if key == ord('q'):
                print("Выход из визуализации.")
                break
            elif key == ord('d'):
                i = min(i + 1, num_pairs - 1)
            elif key == ord('a'):
                i = max(i - 1, 0)

        cv2.destroyAllWindows()
    
    def get_first_valid_frame(self) -> Tuple[int, np.ndarray, List[cv2.KeyPoint], np.ndarray]:
        for i, img in enumerate(self.frames):
            kp, des = self.detect_and_compute(img)
            if des is not None and len(kp) > 0:
                return i, img, kp, des
        raise RuntimeError("Не найдено ни одного кадра с валидными фичами.")
    
    def scale_translation_by_triangulation(self, R, t, pts1, pts2):
        """
        Масштабирует вектор t, используя триангуляцию точек.
        """
        T_temp = np.eye(4)
        T_temp[:3, :3] = R
        T_temp[:3, 3] = t.ravel()

        X_world = self.triangulate_points(pts1, pts2, np.eye(4), T_temp)

        if X_world is None or len(X_world) < 10:
            print("[Scale] Недостаточно точек для оценки масштаба.")
            return t

        depths = np.linalg.norm(X_world, axis=1)
        median_depth = np.median(depths)

        print(f"[Scale] Кол-во точек для оценки: {len(X_world)}")
        print(f"[Scale] Глубина точек: mean={depths.mean():.3f}, median={median_depth:.3f}, min={depths.min():.3f}, max={depths.max():.3f}")
        print(f"[Scale] Норма исходного t: {np.linalg.norm(t):.3f}")

        if median_depth > 1e-6:
            scale_factor = 1.0 / median_depth
            t_scaled = t * scale_factor
            print(f"[Scale] Масштаб применён: factor={scale_factor:.3f}, норма нового t={np.linalg.norm(t_scaled):.3f}")
        else:
            t_scaled = t
            print("[Scale] Ошибка: median_depth слишком мала, масштаб не изменён")

        return t_scaled

    def bundle_adjustment(self, last_frame_size: int = 2):
        if len(self.keyframes) < 2 or len(self.mappoints) < 10:
            print("[BA] Недостаточно данных для оптимизации")
            return

        recent_kfs = self.keyframes[-last_frame_size:]
        print(f"[BA] Оптимизация для {len(recent_kfs)} KeyFrame")

        for kf in recent_kfs:
            obj_points = []
            img_points = []

            for mp in self.mappoints:
                for obs in mp.observations:
                    if obs[0] == kf.id:
                        obj_points.append(mp.coord)
                        img_points.append(obs[2])

            if len(obj_points) < 6:
                print(f"[BA] KeyFrame {kf.id}: мало точек ({len(obj_points)})")
                continue

            obj_points = np.array(obj_points, dtype=np.float64)
            img_points = np.array(img_points, dtype=np.float64)

            # начальная оценка из текущей позы
            R_init = kf.pose[:3, :3]
            t_init = kf.pose[:3, 3]
            rvec_init, _ = cv2.Rodrigues(R_init)
            tvec_init = t_init.reshape(-1, 1)

            success, rvec, tvec = cv2.solvePnP(
                obj_points,
                img_points,
                self.K,
                None,
                rvec_init,
                tvec_init,
                useExtrinsicGuess=True,
                flags=cv2.SOLVEPNP_ITERATIVE
            )

            if success:
                R_opt, _ = cv2.Rodrigues(rvec)
                T_opt = np.eye(4)
                T_opt[:3, :3] = R_opt
                T_opt[:3, 3] = tvec.ravel()
                kf.pose = T_opt
                self.poses[kf.id] = T_opt

                print(f"[BA] KeyFrame {kf.id}: поза обновлена")
            else:
                print(f"[BA] KeyFrame {kf.id}: оптимизация не удалась")
        
    def filter_mappoints_depth(
        self,
        z_min: float = 0.1,
        z_max: float = 100.0,
        x_range: tuple = None,
        y_range: tuple = None
    ):
        """
        Удаляет некорректные или слишком близкие/далёкие MapPoints после инициализации.
        """

        before = len(self.mappoints)
        filtered = []

        for mp in self.mappoints:
            coord = mp.coord

            # Проверка на NaN/Inf
            if not np.isfinite(coord).all():
                continue

            # Фильтрация по глубине
            if coord[2] < z_min or coord[2] > z_max:
                continue

            # Фильтрация по X, Y (если заданы)
            if x_range and not (x_range[0] <= coord[0] <= x_range[1]):
                continue
            if y_range and not (y_range[0] <= coord[1] <= y_range[1]):
                continue

            filtered.append(mp)

        self.mappoints = filtered
        after = len(self.mappoints)

        print(f"[Filter] MapPoints до фильтрации: {before}, после: {after}")
        print(f"[Filter] Удалено {before - after} точек")
    
    def init_map(self):
        print("[Init] Запуск инициализации карты...")

        start_idx, img1, kp1, des1 = self.get_first_valid_frame()
        print(f"[Init] Первый кадр: #{start_idx}, найдено {len(kp1)} ключевых точек.")

        T_1 = np.eye(4, dtype=np.float64)

        for i in range(start_idx + 1, len(self.frames)):
            img2 = self.frames[i]
            kp2, des2 = self.detect_and_compute(img2)

            if des2 is None or len(kp2) == 0:
                print(f"[Init:{i}] Нет валидных дескрипторов, пропускаем кадр.")
                continue

            kp1, kp2, matches, pts1, pts2 = self.get_matches(kp1, des1, kp2, des2)
            print(f"[Init:{i}] Сопоставлено {len(matches)} фич между кадрами {i - 1} и {i}")
            if len(matches) < self.params.get("min_matches", 50):
                print(f"[Init:{i}] Недостаточно хороших матчей: {len(matches)}, пробуем следующий кадр.")
                continue
            
            mean_parallax = np.mean(np.linalg.norm(pts1 - pts2, axis=1))   
            print(f"[Init:{i}] Средний параллакс: {mean_parallax:.2f}")         
            if mean_parallax < self.params.get("min_parallax", 5.0):
                print(f"[Init:{i}] Малый параллакс ({mean_parallax:.2f}) – пробуем следующий кадр.")
                continue

            T_2 = self.estimate_pose_opencv(pts1, pts2)           
            if T_2 is None:
                print(f"[Init:{i}] Не удалось оценить позу, пробуем следующий кадр.")
                continue   
            
            R = T_2[:3, :3]
            t = T_2[:3, 3]
            # t = self.scale_translation_by_triangulation(R, t, pts1, pts2)
            t = t / np.linalg.norm(t)
            T_2[:3, 3] = t


            X_world = self.triangulate_points(pts1, pts2, T_1, T_2)
            if X_world is None or len(X_world) < 50:
                print(f"[Init:{i}] Триангуляция не дала достаточно точек.")
                continue

            self.add_keyframe(start_idx, img1, kp1, des1, T_1)
            self.add_keyframe(i, img2, kp2, des2, T_2)
            print(f"[Init:{i}] Добавлены KeyFrame #{start_idx} и #{i}")

            for j, p in enumerate(X_world):
                desc = des2[matches[j].trainIdx]
                self.add_mappoint(p, i, matches[j].trainIdx, tuple(pts2[j]), desc)

            self.poses.append(T_1)
            self.poses.append(T_2)

            print(f"[Init:{i}] Cмещение: {np.linalg.norm(t):.3f}")
            print(f"[Init:{i}] Инициализация завершена:")
            print(f"  KeyFrames: {len(self.keyframes)}")
            print(f"  MapPoints: {len(self.mappoints)}")
            
            self.bundle_adjustment()
            
            z_min = self.params.get("filter_z_min", 0.1)
            z_max = self.params.get("filter_z_max", 150.0)
            x_range = self.params.get("filter_x_range", (-100, 100))
            y_range = self.params.get("filter_y_range", (-100, 100))

            self.filter_mappoints_depth(
                z_min=z_min,
                z_max=z_max,
                x_range=x_range,
                y_range=y_range
            )
            
            t0 = self.poses[0][:3, 3]
            t1 = self.poses[1][:3, 3]

            print(f"[Init:{i}] Поза камеры 0: X={t0[0]:.3f} Y={t0[1]:.3f} Z={t0[2]:.3f}")
            print(f"[Init:{i}] Поза камеры 1: X={t1[0]:.3f} Y={t1[1]:.3f} Z={t1[2]:.3f}")
            
            return i, img2, kp2, des2

        raise RuntimeError("[Init] Не удалось инициализировать карту.")
    
    def select_mappoint_candidates(self, pose: np.ndarray, image_shape: Tuple[int, int]):
        if pose is None:
            if len(self.poses) == 0:
                return []
            pose = self.poses[-1]

        h, w = image_shape
        candidates = []

        count_invalid = 0
        count_behind = 0
        count_outside = 0

        print(f"[select_mappoint_candidates] Поза камеры (t) = {pose[:3, 3]}")
        print(f"[select_mappoint_candidates] Всего MapPoints = {len(self.mappoints)}")

        # Инверсия pose (camera→world -> world→camera)
        pose_inv = np.linalg.inv(pose)
        K = self.K

        for i, mp in enumerate(self.mappoints):
            if mp.descriptor is None or not np.all(np.isfinite(mp.coord)):
                count_invalid += 1
                continue

            Pw_hom = np.append(mp.coord, 1.0)
            Pc = pose_inv @ Pw_hom

            if Pc[2] <= 0:
                count_behind += 1
                continue

            u = K[0, 0] * (Pc[0] / Pc[2]) + K[0, 2]
            v = K[1, 1] * (Pc[1] / Pc[2]) + K[1, 2]

            if u < 0 or u >= w or v < 0 or v >= h:
                count_outside += 1
                continue

            candidates.append((i, mp.descriptor))

        print(f"[select_mappoint_candidates] Неиспользованы (NaN/нет дескриптора): {count_invalid}")
        print(f"[select_mappoint_candidates] За камерой: {count_behind}")
        print(f"[select_mappoint_candidates] Вне кадра: {count_outside}")
        print(f"[select_mappoint_candidates] Выбрано кандидатов: {len(candidates)}")

        return candidates

    def visualize_map_and_camera(self, pose):
        import open3d as o3d
        import numpy as np

        if len(self.mappoints) == 0:
            print("Нет MapPoints для визуализации")
            return

        h, w = 720, 1280  # либо подставь реальные размеры

        points = []
        colors = []

        pose_inv = np.linalg.inv(pose)
        K = self.K

        for mp in self.mappoints:
            if not np.all(np.isfinite(mp.coord)):
                continue

            Pw_hom = np.append(mp.coord, 1.0)
            Pc = pose_inv @ Pw_hom

            if Pc[2] > 0:
                # Проекция в пиксели
                u = K[0, 0] * (Pc[0] / Pc[2]) + K[0, 2]
                v = K[1, 1] * (Pc[1] / Pc[2]) + K[1, 2]
                if 0 <= u < w and 0 <= v < h:
                    color = [0, 1, 0]  # зелёный (в кадре)
                else:
                    color = [1, 0, 0]  # красный (вне кадра)
            else:
                color = [1, 0, 0]      # красный (за камерой)

            points.append(mp.coord)
            colors.append(color)

        points = np.array(points)
        colors = np.array(colors)

        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)
        pcd.colors = o3d.utility.Vector3dVector(colors)

        frustum = self.create_camera_frustum(scale=5.0)
        frustum.transform(pose)

        cam_center = o3d.geometry.TriangleMesh.create_sphere(radius=0.2)
        cam_center.translate(pose[:3, 3])
        cam_center.paint_uniform_color([1, 0.7, 0])

        o3d.visualization.draw_geometries([pcd, frustum, cam_center],
                                        window_name="MapPoints and Camera",
                                        width=1280, height=720)
        
    def create_camera_frustum(self, scale: float = 0.05) -> o3d.geometry.LineSet:
        """
        Создаёт 3D-модель пирамиды камеры для визуализации в open3d.
        scale – масштаб фрустума
        """
        points = np.array([
            [0, 0, 0],
            [ scale,  scale,  scale * 2],
            [ scale, -scale,  scale * 2],
            [-scale, -scale,  scale * 2],
            [-scale,  scale,  scale * 2],
        ])
        lines = [
            [0, 1], [0, 2], [0, 3], [0, 4],
            [1, 2], [2, 3], [3, 4], [4, 1]
        ]
        frustum = o3d.geometry.LineSet()
        frustum.points = o3d.utility.Vector3dVector(points)
        frustum.lines = o3d.utility.Vector2iVector(lines)
        frustum.paint_uniform_color([0.0, 0.6, 1.0])
        return frustum
    
    def track_frame(self, img, kp, des):

        if len(self.mappoints) == 0 or des is None or len(kp) == 0:
            print("[TrackFrame] Нет MapPoints или дескрипторов.")
            return None

        # 1. Выбор кандидатов
        pose_guess = self.poses[-1] if len(self.poses) > 0 else np.eye(4)
        pose = self.poses[-1]  # или нужный KeyFrame.pose
        # self.visualize_map_and_camera(pose)
        candidates = self.select_mappoint_candidates(pose_guess, img.shape)

        if not candidates:
            print("[TrackFrame] Нет кандидатов для трекинга.")
            return None
        
        # 2. Сопоставление дескрипторов
        des_mps = np.array([desc for _, desc in candidates])
        mp_indices = [idx for idx, _ in candidates]

        matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
        knn_matches = matcher.knnMatch(des_mps, des, k=2)

        # Ratio test
        ratio_thresh = self.params.get("ratio_thresh", 0.6)
        good_matches = []
        matches_map = []

        for pair in knn_matches:
            if len(pair) < 2:
                continue
            m, n = pair
            if m.distance < ratio_thresh * n.distance:
                mp_index = mp_indices[m.queryIdx]
                matches_map.append((mp_index, m.trainIdx))
                good_matches.append(m)

        print(f"[TrackFrame] Совпадений после ratio test: {len(good_matches)}")

        # 3. Формирование пар 3D–2D для PnP
        object_points = np.array([self.mappoints[mp_i].coord for mp_i, _ in matches_map])
        image_points = np.array([kp[kp_i].pt for _, kp_i in matches_map])
    
        
        if object_points.shape[0] < 4:
            print("[TrackFrame] Недостаточно точек для PnP.")
            return None

        # 4. Решение PnP
        success, rvec, tvec, inliers = cv2.solvePnPRansac(
            object_points,
            image_points,
            self.K,
            None,
            iterationsCount=1000,
            reprojectionError=8.0,
            confidence=0.99
        )

        if not success or inliers is None or len(inliers) < 6:
            print("[TrackFrame] PnP не удалось.")
            return None

        # 5. Построение матрицы позы
        R, _ = cv2.Rodrigues(rvec)
        T = np.eye(4)
        T[:3, :3] = R
        T[:3, 3] = tvec.ravel()

        print(f"[TrackFrame] PnP успешен. Inliers: {len(inliers)}")
        print(f"[TrackFrame] Поза камеры: X={T[0,3]:.3f} Y={T[1,3]:.3f} Z={T[2,3]:.3f}")

        # Дальнейшие шаги: добавление наблюдений, проверка на KeyFrame
        pass
    
    def find_mappoint_for_keypoint(self, frame_id: int, kp_idx: int):
        for mp in self.mappoints:
            for obs in mp.observations:
                if obs[0] == frame_id and obs[1] == kp_idx:
                    return mp
        return None
    
    def create_new_keyframe(self, frame_id, img, kp, des, pose):
        print(f"[KeyFrame] Добавлен новый KeyFrame #{frame_id}")
        self.add_keyframe(frame_id, img, kp, des, pose)

        total_new_points = 0

        for prev_kf in self.keyframes[-3:]:  # ищем совпадения с последними 3 KeyFrames
            kp1, kp2, matches, pts1, pts2 = self.get_matches(
                prev_kf.keypoints, prev_kf.descriptors, kp, des
            )

            print(f"[KeyFrame] Матчей с предыдущим KeyFrame #{prev_kf.id}: {len(matches)}")

            if len(matches) < 8:
                continue

            X_world = self.triangulate_points(pts1, pts2, prev_kf.pose, pose)
            if X_world is None:
                continue

            added = 0
            for j, p in enumerate(X_world):
                kp_idx = matches[j].trainIdx

                # Проверяем, не связан ли уже этот keypoint с MapPoint
                if self.find_mappoint_for_keypoint(frame_id, kp_idx) is not None:
                    continue

                desc = des[kp_idx]
                self.add_mappoint(
                    coord_3d=p,
                    frame_id=frame_id,
                    kp_idx=kp_idx,
                    pt2d=tuple(pts2[j]),
                    descriptor=desc
                )
                added += 1

            total_new_points += added
            print(f"[KeyFrame] Добавлено {added} новых MapPoints из пары с KeyFrame #{prev_kf.id}")

        print(f"[KeyFrame] Всего добавлено {total_new_points} новых MapPoints для KeyFrame #{frame_id}")

    def need_new_keyframe(self, T_curr: np.ndarray, keypoints: List[cv2.KeyPoint]) -> bool:
        # Параметры из self.params
        min_tracked = self.params.get("min_tracked_points", 30)
        min_interval = self.params.get("keyframe_interval", 10)
        translation_thresh = self.params.get("keyframe_translation_thresh", 0.2)

        if len(self.keyframes) == 0:
            print("[KeyFrameDecision] Нет KeyFrames – добавляем первый.")
            return True

        # Считаем количество точек, которые трекаются в текущем кадре
        tracked_points = sum(
            1
            for mp in self.mappoints
            for obs in mp.observations
            if obs[0] == len(self.poses) - 1
        )
        print(f"[KeyFrameDecision] Трек точек: {tracked_points}")

        # Проверяем минимальный интервал между KeyFrames
        frames_since_last_kf = len(self.poses) - 1 - self.keyframes[-1].id
        print(f"[KeyFrameDecision] Кадров с последнего KeyFrame: {frames_since_last_kf}")

        if frames_since_last_kf < min_interval:
            print(f"[KeyFrameDecision] Ещё не прошло {min_interval} кадров – KeyFrame не создаём.")
            return False

        # Если трекается мало точек – добавляем KeyFrame
        if tracked_points < min_tracked:
            print("[KeyFrameDecision] Мало точек – добавляем KeyFrame.")
            return True

        # Проверяем смещение камеры
        last_kf_pose = self.keyframes[-1].pose
        delta = np.linalg.norm(T_curr[:3, 3] - last_kf_pose[:3, 3])
        print(f"[KeyFrameDecision] Смещение камеры: {delta:.3f}")

        if delta > translation_thresh:
            print("[KeyFrameDecision] Камера сильно сместилась – добавляем KeyFrame.")
            return True

        print("[KeyFrameDecision] Новый KeyFrame не требуется.")
        return False

    def visualize_keypoints(self, img, kp, window_name="Keypoints"):
        img_vis = cv2.drawKeypoints(
            img, kp, None,
            color=(0, 255, 0),
            flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS
        )
        cv2.imshow(window_name, img_vis)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    
    def run(self):

        if len(self.frames) < 2:
            raise ValueError("Недостаточно кадров для запуска одометрии.")
        
        idx, img1, kp1, des1 = self.init_map()

        # for i in range(idx + 1, len(self.frames)):
        for i in range(idx + 1, 4):
            print(f"Обработка кадра {i} из {len(self.frames)}...")
            img2 = self.frames[i]
            kp2, des2 = self.detect_and_compute(img2)
            
            # self.visualize_keypoints(img2, kp2)

            if des2 is None or len(kp2) == 0:
                continue

            # Трекинг по PnP
            T_curr = self.track_frame(img2, kp2, des2)
            
            # if T_curr is None:
            #     # fallback на get_matches, если PnP не сработал
            #     kp1, kp2, matches, pts1, pts2 = self.get_matches(kp1, des1, kp2, des2)
            #     if len(matches) >= 8:
            #         T_curr = self.estimate_pose_custom(pts1, pts2)
            #     else:
            #         continue

            # self.poses.append(T_curr)

            # # Решение о добавлении нового KeyFrame
            # if self.need_new_keyframe(T_curr, kp2):
            #     self.create_new_keyframe(i, img2, kp2, des2, T_curr)

            img1, kp1, des1 = img2, kp2, des2

In [56]:
import cv2
from pathlib import Path

VIDEO_PATH = DATA_PATH / 'KITTI_sequence_1/image_l'
CALIBRATION_PATH = DATA_PATH / 'KITTI_sequence_1/calib.txt'
TARGET_FPS = 2.0

if not VIDEO_PATH.exists():
    raise FileNotFoundError(f'Видео не найдено по пути: {VIDEO_PATH.resolve()}')

if not CALIBRATION_PATH.exists():
    raise FileNotFoundError(f'Калибровочный файл не найден: {CALIBRATION_PATH.resolve()}')

params = {
    "target_fps": TARGET_FPS,  # частота обработки кадров
    
    # ORB-параметры
    "nfeatures": 400,           # [default=500] больше фич — выше плотность покрытия сцены
    "fastThreshold":20,          # [default=20] ниже порог — чувствительнее к слабым углам
    "edgeThreshold": 5,         # [default=31] позволяет детектировать ближе к краям кадра
    "scaleFactor": 1.2,          # [default=1.2] масштаб между уровнями пирамиды (меньше — медленнее, но точнее)
    "nlevels": 8,               # [default=8] число уровней пирамиды — выше = лучше устойчивость к масштабу

    # Фильтрация матчей по дистанции (Lowe’s ratio test)
    "ratio_thresh": 0.5,         # [default=0.75] строгая фильтрация: оставлять только уверенные совпадения
    "max_match_distance": 200,
    
    # Дополнительно для сетки
    "use_grid": True,       # включение/выключение сетки
    "grid_rows": 4,
    "grid_cols": 6,

    "flann_table_number": 10,
    "flann_key_size": 14,
    "flann_multi_probe_level": 2,
    "flann_checks": 50
}


vo = VisualOdometry(
    video_path=VIDEO_PATH,
    calibration_file=CALIBRATION_PATH,
    **params
)


[KITTI] Загружено 51 кадров из image_l


In [57]:
# from IPython.display import display, clear_output
# from PIL import Image
# import time
# import cv2

# num_frames_to_show = int(TARGET_FPS * 30)
# frames_to_show = vo.frames[:num_frames_to_show]

# for frame in frames_to_show:
#     rgb_frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
#     clear_output(wait=True)
#     display(Image.fromarray(rgb_frame))
#     time.sleep(1 / TARGET_FPS / 2)

In [58]:
vo.clear_data()
vo.run()

[Init] Запуск инициализации карты...
[Debug] Найдено 6268 ключевых точек
[Init] Первый кадр: #0, найдено 6268 ключевых точек.
[Debug] Найдено 6467 ключевых точек
[Init:1] Сопоставлено 369 фич между кадрами 0 и 1
[Init:1] Средний параллакс: 7.08
[Init:1] Добавлены KeyFrame #0 и #1
[Init:1] Cмещение: 1.000
[Init:1] Инициализация завершена:
  KeyFrames: 2
  MapPoints: 329
[BA] Оптимизация для 2 KeyFrame
[BA] KeyFrame 0: мало точек (0)
[BA] KeyFrame 1: поза обновлена
[Filter] MapPoints до фильтрации: 329, после: 291
[Filter] Удалено 38 точек
[Init:1] Поза камеры 0: X=0.000 Y=0.000 Z=0.000
[Init:1] Поза камеры 1: X=2.693 Y=-2.646 Z=23.300
Обработка кадра 2 из 51...
[Debug] Найдено 6368 ключевых точек
[select_mappoint_candidates] Поза камеры (t) = [ 2.69262051 -2.64575406 23.3001648 ]
[select_mappoint_candidates] Всего MapPoints = 291
[select_mappoint_candidates] Неиспользованы (NaN/нет дескриптора): 0
[select_mappoint_candidates] За камерой: 84
[select_mappoint_candidates] Вне кадра: 53
[se

In [59]:
# vo.debug_check_coordinate_system()

In [60]:
# vo.visualize_keypoints_pairwise()

In [61]:
# vo.visualize_matches()

In [62]:
# import open3d as o3d
# import numpy as np

# def visualize_full_map(vo):
#     if len(vo.mappoints) == 0:
#         print("Нет MapPoints для отображения.")
#         return

#     # 1. Создаём облако точек
#     points = np.array([mp.coord for mp in vo.mappoints])
#     pcd = o3d.geometry.PointCloud()
#     pcd.points = o3d.utility.Vector3dVector(points)
#     pcd.paint_uniform_color([1.0, 0.0, 0.0])  # красные точки

#     # 2. Создаём список фрустумов для всех поз камеры
#     frustums = []
#     for i, pose in enumerate(vo.poses):
#         frustum = vo.create_camera_frustum(scale=0.05)
#         frustum.transform(pose)
#         frustums.append(frustum)

#     # 3. Создаём линию траектории
#     trajectory_points = [pose[:3, 3] for pose in vo.poses]
#     trajectory_lines = [[i, i+1] for i in range(len(trajectory_points)-1)]
#     trajectory = o3d.geometry.LineSet()
#     trajectory.points = o3d.utility.Vector3dVector(trajectory_points)
#     trajectory.lines = o3d.utility.Vector2iVector(trajectory_lines)
#     trajectory.paint_uniform_color([0.0, 1.0, 0.0])  # зелёная траектория

#     # 4. Отображаем всё вместе
#     o3d.visualization.draw_geometries([pcd, trajectory, *frustums],
#                                       window_name="Полная карта и траектория",
#                                       width=1280, height=720)


# visualize_full_map(vo)

In [63]:
import open3d as o3d
import numpy as np

def create_camera_frustum(scale: float = 0.05) -> o3d.geometry.LineSet:
    """
    Создаёт 3D-модель пирамиды камеры для визуализации в open3d.
    scale – масштаб фрустума
    """
    points = np.array([
        [0, 0, 0],
        [ scale,  scale,  scale * 2],
        [ scale, -scale,  scale * 2],
        [-scale, -scale,  scale * 2],
        [-scale,  scale,  scale * 2],
    ])
    lines = [
        [0, 1], [0, 2], [0, 3], [0, 4],
        [1, 2], [2, 3], [3, 4], [4, 1]
    ]
    frustum = o3d.geometry.LineSet()
    frustum.points = o3d.utility.Vector3dVector(points)
    frustum.lines = o3d.utility.Vector2iVector(lines)
    frustum.paint_uniform_color([0.0, 0.6, 1.0])
    return frustum


def check_and_visualize_mappoints(vo):
    """
    Проверяет и визуализирует 3D MapPoints после инициализации.
    vo: экземпляр класса VisualOdometry
    """
    if not vo.mappoints:
        print("[Check] Нет MapPoints для отображения.")
        return

    coords = np.array([mp.coord for mp in vo.mappoints if np.all(np.isfinite(mp.coord))])

    if coords.size == 0:
        print("[Check] Все точки невалидны.")
        return

    # Статистика
    depths = coords[:, 2]
    print(f"[Check] Всего точек: {len(coords)}")
    print(f"[Check] Глубина Z: mean={depths.mean():.3f}, median={np.median(depths):.3f}, min={depths.min():.3f}, max={depths.max():.3f}")

    print(f"[Check] X: mean={coords[:,0].mean():.3f}, min={coords[:,0].min():.3f}, max={coords[:,0].max():.3f}")
    print(f"[Check] Y: mean={coords[:,1].mean():.3f}, min={coords[:,1].min():.3f}, max={coords[:,1].max():.3f}")

    print("[Check] Примеры точек:")
    for i in range(min(5, len(coords))):
        print(f"   {coords[i]}")

    # Создаём PointCloud для Open3D
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(coords)
    pcd.paint_uniform_color([1.0, 0.0, 0.0])  # красные точки

    # Добавляем камеры (если есть позы)
    geometries = [pcd]
    for pose in vo.poses:
        frustum = create_camera_frustum(scale=5).transform(pose)
        geometries.append(frustum)

    # Визуализация
    o3d.visualization.draw_geometries(geometries, window_name="MapPoints после инициализации")

# check_and_visualize_mappoints(vo)